# XGBoost Model

## Dependancies

In [1]:
import os
import sys
import csv

import pandas as pd
#pd.set_option('display.max_rows', None)
# pd.options.display.float_format = '{:, .2f}'.format
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns', 100)

import numpy as np
from numpy import save, load
from numpy import savez_compressed
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
import copy
import pickle

#from scipy.misc import comb, logsumexp
from sklearn.manifold import TSNE #a tool to visualize high dimensional data
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD # dimensionality reduction using truncated SVD (AKA LSA)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.collocations import *
import string #python module
import re # python regex module
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

np.random.seed(0)

from sklearn.preprocessing import normalize
from functools import reduce

# XGBoost

Changed to using npz instead of pickle after finding npz average save/load times are shorter than pickle.

In [4]:
# load data

from numpy import load
target_y = load('model_target_data.npz')
target_y = target_y['arr_0']
target_y = np.ravel(target_y)
print(target_y.shape)

features_x =  load('model_data.npz')
features_x = features_x['arr_0']
print(features_x.shape)

(75385,)
(75385, 836)


In [3]:
os.path.getsize('model_data.npz')

463657257

In [5]:
%%time

# instantiate XGBoost classifier
boost_clf = xgb.XGBClassifier(verbosity = 2, max_depth = 6, tree_method= 'gpu_hist' ,predictor = 'gpu_predictor')

## use stratefied kfold for classification task
kfold = StratifiedKFold(n_splits=10)

# get predictions
# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
boost_scores = cross_val_predict(boost_clf, features_x, target_y, cv=kfold)

CPU times: user 45.2 s, sys: 13.5 s, total: 58.8 s
Wall time: 1min


In [10]:
from score import report_score, LABELS, score_submission
predicted = [LABELS[int(a)] for a in boost_scores]
actual = [LABELS[int(a)] for a in target_y]
fold_score, _ = score_submission(actual, predicted)
max_fold_score, _ = score_submission(actual, actual)
score = fold_score/max_fold_score

best_score = 0
best_fold = None
    
#print("Score for fold "+ str(fold) + " was - " + str(score))
if score > best_score:
    best_score = score
    best_fold = boost_scores

In [18]:
report_score(actual,predicted)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |   4327    |    171    |   1030    |    53     |
-------------------------------------------------------------
| disagree  |    442    |    731    |    323    |    41     |
-------------------------------------------------------------
|  discuss  |    354    |    37     |   12845   |    137    |
-------------------------------------------------------------
| unrelated |    23     |     1     |    212    |   54658   |
-------------------------------------------------------------
Score: 32156.75 out of 34214.5	(93.98573704131289%)


93.98573704131289